# 2. Sentiment Analysis using Neural Network

In class, we learned sequence representation and how language models are developed for different tasks. In this assignment, we will implement two neural network models for sentiment analysis task using IMDB dataset. Sentiment analysis in Natural Language Processing (NLP) is a task that involves classifying sentences or text into different categories based on the sentiment expressed. It aims to determine whether the sentiment of the text is positive, or negative. This analysis helps in understanding the overall opinion or emotion conveyed by the text.

Please note that this assignment is built and tested under *Google Colaboratory*. If you work on a local machine, you need to handle version issue on your own. Please complete the given jupyter notebook file and submit it along with your answer to this latex file.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Data Preprocessing

In this assignment, we will use TorchText package to deal with the data. There are a few ways to process the data, but this package makes this procedure much easier. Now we will go over the next steps with TorchText:

- Preprocessing
- Split into train and test set
- Build dataset
- Building vocabulary
- Batching the data

Once you load the data, there will be two columns, review comment and sentiment label (1 for positive and 0 for negative).
Out of 50000 sample data, we will use 30000 as our training set and rest of the data as our test set.
You can refer to [here](https://torchtext.readthedocs.io/en/latest/index.html) for more information on the TorchText.

In [2]:
# !pip install torchtext==0.6.0

In [3]:
import urllib.request
import pandas as pd
from torchtext import data, datasets
from torchtext.vocab import Vocab
from torchtext.data import TabularDataset
from torchtext.data import Iterator


In [4]:
# load data
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")


('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7feee75c9b90>)

In [5]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

review  sentiment
0  My family and I normally do not watch local mo...          1
1  Believe it or not, this was at one time the wo...          0
2  After some internet surfing, I found the "Home...          0
3  One of the most unheralded great works of anim...          1
4  It was the Sixties, and anyone with long hair ...          0

In [6]:
print('Total number of sample data: {}'.format(len(df)))

Total number of sample data: 50000


In [7]:
train_df = df[:30000]
test_df = df[30000:]

train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [8]:
# data preprocessing
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=100)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

In [9]:
# build dataset
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [10]:
print('number of training data : {}'.format(len(train_data)))
print('number of test data : {}'.format(len(test_data)))

number of training data : 30000
number of test data : 20000


In [11]:
print(train_data.fields.items())

dict_items([('text', <torchtext.data.field.Field object at 0x7feee7ebc9d0>), ('label', <torchtext.data.field.Field object at 0x7fefd0148a50>)])


In [12]:
print(train_data[0].text)
print(train_data[0].label)

['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me', 'dabbi

In [13]:
print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [14]:
# build vocabulary
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [15]:
print('size of the vocabulary : {}'.format(len(TEXT.vocab))) # includes <unk> and <pad>

size of the vocabulary : 10002


In [16]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7feee6953610>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'it': 11, 'that': 12, '/><br': 13, 'was': 14, 'as': 15, 'with': 16, 'for': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'are': 22, 'not': 23, 'you': 24, 'film': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'who': 35, 'all': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'about': 42, "it's": 43, 'her': 44, 'if': 45, 'some': 46, 'out': 47, 'what': 48, 'very': 49, 'when': 50, 'more': 51, 'there': 52, 'would': 53, 'even': 54, 'my': 55, 'good': 56, 'she': 57, 'their': 58, 'only': 59, 'no': 60, 'really': 61, 'had': 62, 'up': 63, 'can': 64, 'which': 65, 'see': 66, 'were': 67, 'than': 68, '-': 69, 'we': 70, 'been': 71, 'into': 72, 'get': 73, 'will': 74, 'much': 75, 'because': 76, 'story': 77, 'how': 78, 'most': 7

In [17]:
# build data loader and batching the data
batch_size = 64
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [18]:
print('number of mini-batches for training data : {}'.format(len(train_loader)))
print('number of mini-batches for test data : {}'.format(len(test_loader)))

number of mini-batches for training data : 469
number of mini-batches for test data : 313


In [19]:
batch = next(iter(train_loader))
print(type(batch))

<class 'torchtext.data.batch.Batch'>


In [20]:
print(batch.text)

tensor([[  34,    2, 4886,  ...,    2, 2459,   11],
        [ 186,  121,    3,  ...,  599,  125,    8],
        [  10,   20, 1243,  ...,    0,  121,    3],
        ...,
        [  50,    9,   84,  ...,   85,   54, 5272],
        [   9,  739, 1916,  ...,    2,    0,    0],
        [  55,  108,  318,  ...,    5, 1009,  231]])


In [30]:
batch.text.shape

torch.Size([64, 100])

In [37]:
batch.text.unsqueeze(0).shape

torch.Size([1, 64, 100])

In [21]:
print(batch.label)

tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1])


## Problem 2-1. Multilayer Perceptron (MLP) (30 pts)

In this question, we are going to implement a simple Multilayer Perceptron (MLP) model to classify IMDB dataset. MLP is the classical type of neural network, and they are comprised of one or more layers of neurons. Data is fed to the input layer, there may be one or more hidden layers providing levels of abstraction, and predictions will be made on the output layer.

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Problem 2-1 (a) (10 pts)

Implement a two-layer fully-connected neural network. Your model should contain an embedding layer to represent a word in a dense vector representation. Use ReLU for activation function.

In [385]:
class Linear(nn.Module):
    def __init__(self, input_size, hidden_size, embed_dim, num_classes):
        super(Linear, self).__init__()
        #################### YOUR CODE (3 lines) #####################
        self.embedding = nn.Embedding(len(TEXT.vocab), embed_dim, padding_idx=1) # [batch, sentence_length, embed_dim]
        self.fc1 = nn.Linear(embed_dim * input_size, hidden_size) # x.reshape(batch_size, -1) : [batch, embed_dim * input_size]
        self.fc2 = nn.Linear(hidden_size, num_classes) # num_classes = 분류하고자 하는 카테고리 갯수, same as output_dim
        ###############################################################

    def forward(self, text):
        #################### YOUR CODE (3-5 lines) #####################
        # hint: you can use F.relu and F.softmax
        x = self.embedding(text)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        preds = F.relu(self.fc2(x))
        ###############################################################
        return preds

### Problem 2-1 (b) (10 pts)

Implement a function to check the accuracy of your model.

In [398]:
def model_accuracy(output, target):
    #################### YOUR CODE (2-3 lines) #####################
    accuracy = 0
    pred = output.argmax(dim=1)
    score = (pred == target).sum()
    accuracy = score / target.shape[0]
    ###############################################################
    return accuracy

### Problem 2-1 (c) (5 pts)

Complete the code below to train the model

In [399]:
def train(model, iterator, optimizer, criterion):

    loss_ep = 0
    acc_ep = 0

    for batch in iterator:
        optimizer.zero_grad()

        #################### YOUR CODE (1-2 lines) #####################
        model.train()
        output = model(batch.text)
        ###############################################################
        loss = criterion(output, batch.label.squeeze())
        acc = model_accuracy(output, batch.label)

        loss.backward()
        optimizer.step()

        loss_ep += loss.item()
        acc_ep += acc.item()

    return loss_ep / len(iterator), acc_ep / len(iterator)

### Problem 2-1 (d) (5 pts)

Complete the code below to evaluate the model

In [400]:
def evaluate(model, iterator, criterion):

    loss_ep = 0
    acc_ep = 0

    model.eval()
    with torch.no_grad():
        for batch in iterator:
            #################### YOUR CODE (1-2 lines) #####################
            model.eval()
            output = model(batch.text)
            ###############################################################
            loss = criterion(output, batch.label)
            acc = model_accuracy(output, batch.label)

            loss_ep += loss.item()
            acc_ep += acc.item()

    return loss_ep / len(iterator), acc_ep / len(iterator)

Now let's see the model performance

In [401]:
def train_the_model(epochs, model, train_loader, valid_loader, optimizer, criterion):

    for epoch in range(epochs):

        train_loss, train_acc = train(model, train_loader, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_loader, criterion)

        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

    torch.save(model.state_dict(), 'saved_weights_linear.pt')

In [402]:
# hyperparameter
num_epochs = 10
lr = 1e-4
max_token_length = 100
hidden_size = 100
embed_dim = 100
seed = 1
num_classes = 2

In [403]:
loss_function = nn.CrossEntropyLoss()
linear_model = Linear(max_token_length, hidden_size, embed_dim, num_classes)
optimizer = torch.optim.Adam(linear_model.parameters(), lr=lr)

train_the_model(num_epochs, linear_model, train_loader, test_loader, optimizer, loss_function)

	Train Loss: 0.688 | Train Acc: 52.65%
	 Val. Loss: 0.673 |  Val. Acc: 57.82%
	Train Loss: 0.568 | Train Acc: 70.79%
	 Val. Loss: 0.668 |  Val. Acc: 60.50%
	Train Loss: 0.395 | Train Acc: 83.16%
	 Val. Loss: 0.709 |  Val. Acc: 61.35%
	Train Loss: 0.242 | Train Acc: 91.84%
	 Val. Loss: 0.777 |  Val. Acc: 61.86%
	Train Loss: 0.135 | Train Acc: 95.98%
	 Val. Loss: 0.845 |  Val. Acc: 62.30%
	Train Loss: 0.077 | Train Acc: 97.87%
	 Val. Loss: 0.932 |  Val. Acc: 62.48%
	Train Loss: 0.043 | Train Acc: 99.12%
	 Val. Loss: 1.004 |  Val. Acc: 62.41%
	Train Loss: 0.020 | Train Acc: 99.82%
	 Val. Loss: 1.072 |  Val. Acc: 62.77%
	Train Loss: 0.010 | Train Acc: 99.95%
	 Val. Loss: 1.135 |  Val. Acc: 62.92%
	Train Loss: 0.006 | Train Acc: 99.99%
	 Val. Loss: 1.196 |  Val. Acc: 63.07%


## Problem 2-2. Convolutional Neural Network (CNN) (20 pts)

Next, we will perform sentimental analysis on the same dataset with Convolutional Neural Network (CNN). In a CNN, text is organised into a matrix, with each row representing a word embedding. The CNN’s convolutional layer scans the text like it would an image, breaks it down into features, and judges whether each feature matches the relevant label or not.
You can refer to [here](https://emnlp2014.org/papers/pdf/EMNLP2014181.pdf) for the use of CNN on text classification.


### Problem 2-2 (a) (15 pts)

Complete the code below for CNN. Again, your model will require an embedding layer to represent into dense vector. After getting embeddings, we will feed the tensors through the convolutional layer, applying the ReLu activation function following the convlutional layers. Then the tensors will be passed through pooling layers. Lastly, apply dropout to the concatenated filter outputs and subsequently pass them through a linear layer to *generate* our predictions.

In [429]:
embedding = nn.Embedding(len(TEXT.vocab), 300) # input_dim, embed_dim = 300
filter_sizes = [3,5,7]
num_filters = 100
convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(filter_size, 300)) for filter_size in filter_sizes])
fc = nn.Linear(len(filter_sizes)*num_filters, num_classes)

In [430]:
x = embedding(batch.text)
print(x.shape) # batch, sen_len, embed_dim
x = x.unsqueeze(1)
x.shape

torch.Size([32, 100, 300])


torch.Size([32, 1, 100, 300])

In [431]:
for c in convs:
    print(c(x).shape)

torch.Size([32, 100, 98, 1])
torch.Size([32, 100, 96, 1])
torch.Size([32, 100, 94, 1])


In [448]:
conv = [c(embedded).squeeze(3) for c in convs]
conv[2].shape # output size of the n-convolutional layer = [batch_size, n_filters, sen_len - filter_sizes[n] + 1]

torch.Size([32, 100, 94])

In [456]:
maxPools = [(F.max_pool1d(conv[i], conv[i].shape[2])) for i in range(len(filter_sizes))]
maxPools[0].squeeze(2).shape

torch.Size([32, 100])

In [457]:
class CNN(nn.Module):
    def __init__(self, input_dim, embed_dim, n_filters, filter_sizes, num_classes, dropout_rate):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim) # input_dim = len(TEXT.vocab)
        # convolutions with different size of filters
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                              out_channels=n_filters,
                                              kernel_size=(filter_size, embed_dim))
                                    for filter_size in filter_sizes])
        # in_channels: input 채널 갯수, ex:RGB = 3
        # out_channels: output 채널 갯수, 즉 convolution filter 수, 값이 클수록 복잡한 특성 학습가능
        # kernel_size: convolution filter 크기 ex: 3x3 filter -> kernelsize = 3 / 여기선 filter_size x embed_dim
        # stride는 기본 1
        self.fc = nn.Linear(len(filter_sizes)*n_filters, num_classes)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, text):
        # text = [batch_size, sen_len]

        embedded = self.embedding(text)
        # embedded = [batch_size, sen_len, embed_dim] 

        embedded = embedded.unsqueeze(1)
        # embedded = [batch_size, 1, sen_len, embed_dim]

        #################### YOUR CODE (4 lines) #####################
        # hint: you can use F.relu, F.max_pool1d, and torch.cat
        
        # After getting embeddings, we will feed the tensors through the convolutional layer, applying the ReLu activation function following the convlutional layers.
        # output size of the n-convolutional layer = [batch_size, n_filters, sen_len - filter_sizes[n] + 1]
        convolutions = [F.relu(conv(embedded).squeeze(3)) for conv in self.convs]
        
        # Then the tensors will be passed through pooling layers.
        # output size of the n-pooling layer = [batch_size, n_filters]
        
        maxPools = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in convolutions]
    
        # Lastly, apply dropout to the concatenated filter outputs and subsequently pass them through a linear layer to generate our predictions.
        # output size of the dropout = [batch size, n_filters * len(filter_sizes)]
        output = self.dropout(torch.cat(maxPools, dim = 1))
        

        ###############################################################
        return output

Now let's see the model performance.

We will use the same function that we used in MLP model for model training and evaluation.
It will take about 30 minutes to run 5 epochs

In [458]:
# hyperparameter
num_epochs = 5
lr = 1e-4
input_dim = len(TEXT.vocab)
embed_dim = 300
num_filters = 100
filter_sizes = [3,5,7]
dropout_rate = 0.25
num_classes = 2

In [460]:
cnn_model = CNN(input_dim, embed_dim, num_filters, filter_sizes, num_classes, dropout_rate)

optimizer = torch.optim.Adam(cnn_model.parameters(), lr=lr)
train_the_model(num_epochs, cnn_model, train_loader, test_loader, optimizer, loss_function)


	Train Loss: 2.780 | Train Acc: 45.11%
	 Val. Loss: 1.430 |  Val. Acc: 50.97%
	Train Loss: 2.300 | Train Acc: 47.86%
	 Val. Loss: 1.409 |  Val. Acc: 53.56%
	Train Loss: 2.227 | Train Acc: 49.26%
	 Val. Loss: 1.341 |  Val. Acc: 53.61%
	Train Loss: 2.229 | Train Acc: 50.21%
	 Val. Loss: 1.383 |  Val. Acc: 55.53%
	Train Loss: 2.202 | Train Acc: 51.78%
	 Val. Loss: 1.383 |  Val. Acc: 54.58%


### Problem 2-2 (b) (5 pts)

Compare the linear model (problem 1) and CNN (problem 2), and discuss their advantages and limitations.

Answer: linear 모델은 속도가 굉장히 빠르지만, ovefitting이 잘 일어날 수 있다는 단점이 있다.
하지만 CNN은 모델이 굉장히 느리지만 더 복잡한 관계에 대해서도 잘 학습이 가능하다. 